In [1]:
import py_planning
py_planning.init()

In [2]:
from IPython.display import IFrame
IFrame('http://127.0.0.1:8000', width="100%", height=650)

In [5]:
from py_planning.data_types import PlannedPath, PlannedState, State
from shapely.geometry import LineString, Point


def get_index_of_closest_point(line: LineString, point: Point):
    closest_point_index = None
    min_distance = float('inf')

    for i, line_point in enumerate(line.coords):
        line_point = Point(line_point)
        distance = point.distance(line_point)
        if distance < min_distance:
            min_distance = distance
            closest_point_index = i

    return closest_point_index


def do_plan(state: State) -> PlannedPath:
    vehicle_pose = state.vehiclePose
    vehicle_pos = Point(vehicle_pose.pos.x, vehicle_pose.pos.y)

    all_centerlines = []
    for centerline in state.lanePath.centerlines:
        all_centerlines.extend(centerline)

    centerline = LineString([(p.x, p.y) for p in all_centerlines])

    closest_index = get_index_of_closest_point(centerline, vehicle_pos)
    current_velocity = vehicle_pose.velocity

    prev_poses_count = 3
    max_poses_count = 50
    first_pose_index = max(closest_index-prev_poses_count, 0)

    planned_states = [
        PlannedState(pos=p, velocity=current_velocity) for p in all_centerlines
    ][first_pose_index:first_pose_index+ max_poses_count]

    #planned_states[3].pos.x += 10

    return PlannedPath(states=planned_states)

py_planning.run_planner(do_plan)

Case failed: Out of road


False